In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

In [2]:
path =   # Enter the directory of your project folder
PATH = path + "/Results"

if not os.path.exists(PATH):
    os.mkdir(PATH) 


In [3]:
df = pd.read_csv(path+"//ToyDataset.csv").drop(['Unnamed: 0'],axis=1)
clientID = df.ClientID.unique()

In [4]:
# separating the living situation and the exit destination into categories
goal = [10,11]
closer = [3,19,20,21,22,23,25,26,28,31]
trans = [1,2,4,5,6,12,13,14,15,18,27,29]
no_progress = [7,16]
hard_to_judge = [8,9,17,24,30,99]

In [5]:
# labelling the data
dest = pd.DataFrame([])
exit = pd.DataFrame([])
clientID = df.ClientID.unique()
for i in range(len(clientID)):
    temp = df[df['ClientID']==clientID[i]] 
    if (temp['LivingSituation'].iloc[0] in goal)  :
        dest = dest.append({"ClientID":clientID[i],"PathLength":len(temp),"entryFrom":'ultimate goal'},ignore_index=True)
    elif temp['LivingSituation'].iloc[0] in closer:
        dest = dest.append({"ClientID":clientID[i],"PathLength":len(temp),"entryFrom":'closer to exit'},ignore_index=True)
    elif temp['LivingSituation'].iloc[0] in trans:
        dest = dest.append({"ClientID":clientID[i],"PathLength":len(temp),"entryFrom":'transitional phase'},ignore_index=True)
    elif temp['LivingSituation'].iloc[0] in no_progress:
        dest = dest.append({"ClientID":clientID[i],"PathLength":len(temp),"entryFrom":'no progress or worse'},ignore_index=True)
    elif temp['LivingSituation'].iloc[0] in hard_to_judge:
        dest = dest.append({"ClientID":clientID[i],"PathLength":len(temp),"entryFrom":'hard to judge'},ignore_index=True)
    
    if temp['Destination'].iloc[-1] in goal:
        exit = exit.append({"ClientID":clientID[i],'exit':"ultimate goal"},ignore_index=True)
    elif temp['Destination'].iloc[-1] in closer:
            exit = exit.append({"ClientID":clientID[i],'exit':"closer to exit"},ignore_index=True)
    elif temp['Destination'].iloc[-1] in trans:
            exit = exit.append({"ClientID":clientID[i],'exit':"transitional phase"},ignore_index=True)
    elif temp['Destination'].iloc[-1] in no_progress:
            exit = exit.append({"ClientID":clientID[i],'exit':"no progress or worse"},ignore_index=True)
    elif temp['Destination'].iloc[-1] in hard_to_judge:
            exit = exit.append({"ClientID":clientID[i],'exit':"hard to judge"},ignore_index=True)

            
data = pd.merge(dest,exit,on = "ClientID")
data.to_csv(PATH + "//Scheme 4 labelled data.csv")

In [7]:
# determining dropout probability trend

data = pd.read_csv(PATH + "//Scheme 4 labelled data.csv").drop("Unnamed: 0", axis=1)

data_drop = pd.DataFrame()
for i in range(len(data)):
    for j in np.arange(data.PathLength.min(),data.PathLength.max()+1):
        if (j<data['PathLength'].iloc[i]):
            data_drop = data_drop.append({"ClientID":data['ClientID'].iloc[i], "PathLength": data['PathLength'].iloc[i],"DropOut":-1,"step":j},ignore_index = True)
        elif (j==data['PathLength'].iloc[i]):
            if (data.iloc[i]['exit'] == "transitional phase") or (data.iloc[i]['exit'] == "no progress or worse") or (data.iloc[i]['exit'] == "hard to judge") or (data.iloc[i]['exit'] == "closer to exit"):
                data_drop = data_drop.append({"ClientID":data['ClientID'].iloc[i], "PathLength": data['PathLength'].iloc[i],"DropOut":1,"step":j},ignore_index = True)
            else:
                data_drop = data_drop.append({"ClientID":data['ClientID'].iloc[i], "PathLength": data['PathLength'].iloc[i],"DropOut":0,"step":j},ignore_index = True)
        else:
            continue

loc = np.sort(data_drop["step"].unique())
data_drop_prob = pd.DataFrame()
for y in loc:
    temp = data_drop[data_drop['step'] == y]
    dropout = len(temp[temp["DropOut"]==1])/len(temp)
    not_dropoutOrReentry = len(temp[temp["DropOut"]==0])/len(temp) 
    reentry = len(temp[temp["DropOut"]==-1])/len(temp) 
    data_drop_prob = data_drop_prob.append({"Path Position":y,"P(dropout)":dropout,"P(success)":not_dropoutOrReentry, "P(reentry)":reentry},ignore_index=True)

data_drop_prob['Dropout_MA'] = data_drop_prob.iloc[:,0].rolling(window=6).mean()
data_drop_prob['Reentry_MA'] = data_drop_prob.iloc[:,1].rolling(window=6).mean()
data_drop_prob['Success_MA'] = data_drop_prob.iloc[:,2].rolling(window=6).mean()

# data_drop_prob['Dropout_MA'] = data_drop_prob['Dropout_MA'].replace({data_drop_prob['Dropout_MA'].min():10**-4}) 
# data_drop_prob['Success_MA'] = data_drop_prob['Success_MA'].replace({data_drop_prob['Success_MA'].min():10**-4, 
#                                                                     data_drop_prob['Success_MA'][18]:10**-4}) 

plt.figure(figsize = (20,10))
# plt.plot(data_drop_prob['Path Position'],data_drop_prob['P(dropout)'],marker = "x",linewidth = 2,label='drop-out')
# plt.plot(data_drop_prob['Path Position'],data_drop_prob['P(success)'],marker = "x",linewidth = 2,label='success')
# plt.plot(data_drop_prob['Path Position'],data_drop_prob['P(reentry)'],marker = "x",linewidth = 2,label='reentry')
plt.plot(data_drop_prob['Path Position'],data_drop_prob['Dropout_MA'],marker = "x",linewidth = 2,label='Dropout')
plt.plot(data_drop_prob['Path Position'],data_drop_prob['Reentry_MA'],marker = "x",linewidth = 2,label='Reentry')
plt.plot(data_drop_prob['Path Position'],data_drop_prob['Success_MA'],marker = "x",linewidth = 2,label='Success')

plt.legend(fontsize=16)
# plt.yscale("log")
# plt.xscale("log")
# plt.xlabel("Path Position",fontsize=16)
# plt.ylabel("Dropout Probabiliy",fontsize=16)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.title("Scheme 4",fontsize = 16)
plt.savefig(PATH+"/Scheme 4 Drop-out probability with reentry_smoothed.png")
plt.close()

In [9]:
# calculating cdf

def cdf_plot(label,pdf,k):
    plt.subplot(4,1,k)
    size = 35
    min_x = 40
    max_x = 0
    target = ["ultimate goal","closer to exit","transitional phase","no progress or worse","hard to judge"]
    index_tup=[(pdf.index.values[i][0],pdf.index.values[i][1]) for i in range(len(pdf.index.values))]    
    for i in target:
        if (label,i) not in index_tup:
            continue
        cdf = np.cumsum(pdf.loc[label,i]/pdf.loc[label,i].sum())
        cdf["ClientID"].plot(figsize = (25,25),marker="o",label=label+"-"+i,linewidth = 4)
        if cdf.index.min()<min_x:
            min_x = cdf.index.min()
        if cdf.index.max()>max_x:
            max_x = cdf.index.max()
    plt.legend(fontsize=size)
    plt.grid()
    plt.tight_layout()
    plt.xlabel('', fontsize=size)
    plt.yticks(fontsize=size)
    plt.xticks(np.arange(min_x,max_x,4),fontsize =size)

data = pd.read_csv(PATH + "//Scheme 4 labelled data.csv").drop("Unnamed: 0", axis=1)
pdf = data.groupby(['entryFrom','exit','PathLength']).count()

initial = ["closer to exit","transitional phase","no progress or worse","hard to judge"]
[cdf_plot(initial[i],pdf,i+1) for i in range(len(initial))]
plt.savefig(PATH+"/Scheme 4 Aggregated CDF.png", bbox_inches = "tight")
plt.close()

No handles with labels found to put in legend.
